<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Persist, Deploy, and Score an SPSS Model to Predict Customer Churn</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook demonstrates how to deploy an SPSS model and score test data. 

Some familiarity with Python is helpful. This notebook is compatible with Python 3.5.

You will use the **Telco Customer Churn** data set which contains anonymous data about customers of a telecommunication company. Use the details of this data set to predict customer churn which is critical to business as it is easier to retain existing customers rather than acquire new ones.

## Learning goals

In this notebook you learn how to:

-  Set up a Watson Machine Learning instance.
-  Deploy the SPSS model online.
-  Use the deployed model to score data.


## Contents

1.	[Set up the environment](#setup)
2.	[Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository](#scoring)
3.	[Summary](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning (WML) Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance (a free plan is offered and information about how to create the instance can be found [here](https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html)).

-   Install [watson-machine-learning-client](https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search) package (not required on Watson Studio).

#### Install the `watson-machine-learning-client` package from pypi.  
**Note:** `watson-machine-learning-client` documentation can be found <a href="http://wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [ ]:
!pip install --upgrade watson-machine-learning-client 

Authenticate the Watson Machine Learning service on the IBM Cloud.

**Tip**: Authentication information (your credentials) can be found in the <a href="https://console.bluemix.net/docs/services/service_credentials.html#service_credentials" target="_blank" rel="noopener no referrer">Service Credentials</a> tab of the service instance that you created on the IBM Cloud. <BR>If you cannot find the **instance_id** field in **Service Credentials**, click **New credential (+)** to generate new authentication information. 

**Action**: Enter your Watson Machine Learning service instance credentials here.

In [3]:
wml_credentials={
    'url': 'https://ibm-watson-ml.mybluemix.net',
    'username': '***',
    'password': '***',
    'instance_id': '***'
}

Import the `watson-machine-learning-client` module and authenticate the service instance.

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [6]:
client = WatsonMachineLearningAPIClient(wml_credentials)

You cand check the details of your Watson Machine Learning service by calling the ``service_instance.details`` attribute.

In [7]:
instance_details = client.service_instance.details

Get a list of stored models in the WML repository.

In [8]:
client.repository.list_models()

------------------------------------  -------------------------------------------  ------------------------  -----------------
GUID                                  NAME                                         CREATED                   FRAMEWORK
4e7d708b-783c-4f82-95c2-90d02957031f  Customer churn Spark model                   2019-03-11T15:27:06.684Z  mllib-2.3
56529f33-9bce-45f5-87f5-b6bc838b88a0  MNIST best model                             2019-03-07T13:37:39.720Z  tensorflow-1.5
a59d7814-b8f9-4e13-b433-253364dec6ff  Customer churn Spark model                   2019-03-07T13:14:14.864Z  mllib-2.3
5fb4a3c0-f18a-4936-ba15-294c13849167  Product line model                           2019-03-06T14:24:29.322Z  mllib-2.3
193f97a4-2a48-4be7-9497-11ccc58aebb3  My cool mnist model                          2019-03-06T12:52:11.179Z  tensorflow-1.5
2dcc72ce-16c2-4ad0-94c8-77f37317b47b  Boston house price prediction                2019-03-06T11:45:30.725Z  scikit-learn-0.19
f7982685-3040-4936-a7a

Get a list of deployed models.

In [9]:
client.deployments.list()

------------------------------------  -------------------------------------------------------  -------  --------------  ------------------------  -----------------  -------------
GUID                                  NAME                                                     TYPE     STATE           CREATED                   FRAMEWORK          ARTIFACT TYPE
dce80b25-3d32-4005-b96b-9d9c80a7239a  MNIST keras deployment                                   online   DEPLOY_FAILURE  2019-03-07T13:37:41.529Z  tensorflow-1.5     model
00edc46c-5df5-482e-936a-4d9f2e5fbc06  Product line model deployment                            online   DEPLOY_SUCCESS  2019-03-06T14:30:35.086Z  mllib-2.3          model
86da6500-18ff-4970-b3e4-7e335d4a08f9  Mnist model deployment                                   online   DEPLOY_SUCCESS  2019-03-06T12:52:13.175Z  tensorflow-1.5     model
d250df1b-6e43-4c69-98d2-10ff25b5d63d  Virtual deployment of Boston model                       virtual  DEPLOY_SUCCESS  2019-03-0

<a id="scoring"></a>
## 2. Persist, deploy, and score the SPSS model in the Watson Machine Learning (WML) repository

In this section, you will learn how to:
- [2.1 Download and save an SPSS model](#save)
- [2.2 Create an online deployment](#create)
- [2.3 Score a test data record](#score)

For more information about REST APIs, please see the <a href="http://watson-ml-api.mybluemix.net/" target="_blank" rel="noopener no referrer">Swagger Documentation</a>.

### 2.1 Download and save an SPSS model<a id="save"></a>

Download the SPSS sample model from the <a href="https://github.com/pmservice/wml-sample-models" target="_blank" rel="noopener no referrer">Git repository</a>.

**Note:** You may need to install the `wget` package. To install the `wget` package, run the following command. 

In [ ]:
!pip install --upgrade wget

In [11]:
# Download the sample SPSS model.
import os
import wget

sample_dir = 'spss_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)

filename=os.path.join(sample_dir, 'customer-satisfaction-prediction.str')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/spss/customer-satisfaction-prediction/model/customer-satisfaction-prediction.str',\
                             out=sample_dir)
print(filename)

spss_sample_model/customer-satisfaction-prediction.str


Save the SPSS sample model in your WML repository.

In [12]:
# Save the SPSS model to the WML repository.
props_spss = {
    client.repository.ModelMetaNames.NAME: 'SPSS model for Churn prediction',
    client.repository.ModelMetaNames.AUTHOR_NAME: 'IBM', 
    client.repository.ModelMetaNames.FRAMEWORK_NAME: 'spss-modeler', 
    client.repository.ModelMetaNames.FRAMEWORK_VERSION: '18.1'
}

model_details = client.repository.store_model(filename, props_spss)

Get details of all the models saved in the repository.

In [13]:
# List models in the repository.
print(model_details)

{'metadata': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd', 'modified_at': '2019-03-12T12:59:13.864Z', 'guid': '93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd', 'created_at': '2019-03-12T12:59:13.801Z'}, 'entity': {'learning_iterations_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd/learning_iterations', 'feedback_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd/feedback', 'author': {'name': 'IBM'}, 'learning_configuration_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd/learning_configuration', 'runtime_environment': 'None Provided', 'latest_version': {'url': 'https://us-south.ml.cloud.ibm.com/v

### 2.2 Create an online deployment<a id="create"></a>

Using the code below to create an online deployment.

In [14]:
model_uid = client.repository.get_model_uid(model_details)
model_deployment = client.deployments.create(model_uid, 'Sample SPSS model deployment')



#######################################################################################

Synchronous deployment creation for uid: '93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS........
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='92991643-a229-4f9e-a70a-3e9de612942e'
------------------------------------------------------------------------------------------------




Now, you can check details of your deployments.

In [15]:
deployment_uid = client.deployments.get_uid(model_deployment)
print(client.deployments.get_details(deployment_uid))

{'metadata': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/deployments/92991643-a229-4f9e-a70a-3e9de612942e', 'modified_at': '2019-03-12T12:59:16.048Z', 'guid': '92991643-a229-4f9e-a70a-3e9de612942e', 'created_at': '2019-03-12T12:59:15.857Z'}, 'entity': {'model_type': 'spss-modeler-18.1', 'runtime_environment': 'None Provided', 'status_details': {'status': 'DEPLOY_SUCCESS'}, 'scoring_url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/deployments/92991643-a229-4f9e-a70a-3e9de612942e/online', 'status': 'DEPLOY_SUCCESS', 'type': 'online', 'name': 'Sample SPSS model deployment', 'deployable_asset': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/b4b6c696-172c-4164-8049-c0b621dbf3c9/published_models/93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd', 'name': 'SPSS model for Churn prediction', 'type': 'model', 'guid': '93638ac0-a8ae-4cf7-a3b5-47e026ad5bcd', 'created_at': '2019-03-12T12:59:15.793Z'},

### 2.3 Score a test data record<a id="score"></a>

Get the scoring endpoint, `scoring_url`, for your deployed model using the code below.

In [ ]:
# Get the scoring endpoint.
scoring_url = client.deployments.get_scoring_url(model_deployment)
print(scoring_url)

Use the client `score` method to score test data about a customer.

In [17]:
scoring_payload = {'fields':['customerID','gender','SeniorCitizen','Partner','Dependents','tenure','PhoneService','MultipleLines','InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract','PaperlessBilling','PaymentMethod','MonthlyCharges','TotalCharges','Churn','SampleWeight'],'values':[['3638-WEABW','Female',0,'Yes','No',58,'Yes','Yes','DSL','No','Yes','No','Yes','No','No','Two year','Yes','Credit card (automatic)',59.9,3505.1,'No',2.768]]}
prediction = client.deployments.score(scoring_url, scoring_payload)

print(prediction)

{'values': [['3638-WEABW', 'No', 'No', 0.0526309571556145]], 'fields': ['customerID', 'Churn', 'Predicted Churn', 'Probability of Churn']}


As you can see, this sample telco customer is satisfied ("Predicted Churn", "No").

<a id="summary"></a>
## 3. Summary     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for SPSS model deployment and scoring. Check out our <a href="https://dataplatform.ibm.com/docs/content/analyze-data/wml-setup.html" target="_blank" rel="noopener no referrer">Online Documentation</a> for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Lukasz Cmielowski**, Ph.D., is an Automation Architect and Data Scientist at IBM with a track record of developing enterprise-level applications that substantially increase the clients' ability to turn data into actionable knowledge.  
**Jihyoung Kim**, Ph.D., is a Data Scientist at IBM who strives to make data science easy for everyone through Watson Studio.

Copyright © 2017-2019 IBM. This notebook and its source code are released under the terms of the MIT License.

<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>